In [1]:
%%script echo skipping
%pip install snowflake-connector-python
%pip install pyarrow

skipping


In [2]:
import os

import snowflake.connector
from dotenv import load_dotenv

/opt/homebrew/lib/python3.9/site-packages/snowflake/connector/options.py:96: UserWarning: You have an incompatible version of 'pyarrow' installed (9.0.0), please install a version that adheres to: 'pyarrow<8.1.0,>=8.0.0; extra == "pandas"'
  warn_incompatible_dep(


In [3]:
load_dotenv()

True

In [4]:
SNOW_USER = os.environ['SNOW_USER']
ACCOUNT = os.environ['ACCOUNT']
PASSWORD = os.environ['PASSWORD']
WAREHOUSE = os.environ['WAREHOUSE']
DATABASE = os.environ['DATABASE']
SCHEMA = os.environ['SCHEMA']

In [5]:
conn = snowflake.connector.connect(
    user= SNOW_USER,
    account= ACCOUNT,
    password= PASSWORD,
    warehouse= WAREHOUSE,
    database= DATABASE,
    schema= SCHEMA,
)
cs = conn.cursor()